In [1]:
%%time
%load_ext autoreload
%autoreload 2

# import sys
import os
import warnings

# from urllib.request import urlretrieve
# from argparse import Namespace
# from importlib import reload


# import numpy as np
# import pandas as pd
import scanpy as sc
# import matplotlib.pyplot as plt
# import seaborn as sns
# import networkx as nx

# from sklearn.decomposition import _nmf as sknmf
from sklearn.exceptions import ConvergenceWarning
# from scipy.stats import rankdata
# from scipy.cluster import hierarchy

from gepdynamics import _utils
# from gepdynamics import _constants
# from gepdynamics import cnmf
# from gepdynamics import pfnmf
from gepdynamics import comparator

# Move to the project's home directory, as defined in _constants
_utils.cd_proj_home()
print(os.getcwd())

/cs/labs/mornitzan/yotamcon/gep-dynamics
CPU times: user 2.87 s, sys: 1.02 s, total: 3.89 s
Wall time: 5.6 s


In [2]:
%%time
results_dir = _utils.set_dir('results')

split_adatas_dir = _utils.set_dir(results_dir.joinpath('marjanovic_mmLungPlate_split'))

k_12 = sc.read_h5ad(split_adatas_dir.joinpath('04_K_12w_ND_GEPs.h5ad'))

k_30 = sc.read_h5ad(split_adatas_dir.joinpath('05_K_30w_ND_GEPs.h5ad'))

kp_12 = sc.read_h5ad(split_adatas_dir.joinpath('06_KP_12w_ND_GEPs.h5ad'))

kp_30 = sc.read_h5ad(split_adatas_dir.joinpath('08_KP_30w_ND_GEPs.h5ad'))

### Comparing Kras 12 and 30 weeks

In [3]:

adata_a = k_12
adata_b = k_30

comparison_dir = results_dir.joinpath(f"{adata_a.uns['sname']}_{adata_b.uns['sname']}")

tst = comparator.Comparator(
    adata_a, adata_a.obsm['usages'], adata_b, comparison_dir, 'torchnmf',
    max_nmf_iter=800, verbosity=1, )

with warnings.catch_warnings():  # supress convergence warning
    warnings.simplefilter(action='ignore', category=ConvergenceWarning)
    tst.extract_geps_on_jointly_hvgs()

tst.decompose_b(repeats=10)

tst.plot_loss_per_cell_histograms(show=False)
tst.plot_usages_clustermaps(show=False)
tst.plot_decomposition_comparisons(show=False)
tst.run_gsea(gprofiler_kwargs=dict(organism='hsapiens', sources=['GO:BP', 'WP', 'REAC', 'KEGG']))

tst.calculate_fingerprints()

tst.print_errors()

tst.save_to_file(comparison_dir.joinpath('comparator.npz'))

/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 410355.6875, # iterations was 50
Decomposing B de-novo
beta 1 loss = 356439.53125, # iterations was 400
beta 1 loss = 349817.53125, # iterations was 460
beta 1 loss = 342226.15625, # iterations was 530
beta 1 loss = 336116.34375, # iterations was 620
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 111 iterations reachederror =  393991.6
repeat 1, after 93 iterations reachederror =  393974.6
repeat 2, after 87 iterations reachederror =  393940.2
repeat 3, after 100 iterations reachederror =  393954.2
repeat 4, after 92 iterations reachederror =  393979.6
repeat 5, after 103 iterations reachederror =  393944.2
repeat 6, after 106 iterations reachederror =  393941.1
repeat 7, after 101 iterations reachederror =  393968.3
repeat 8, after 109 iterations reachederror =  393944.0
repeat 9, after 95 iterations reachederror =  393966

KeyboardInterrupt: 